<a href="https://colab.research.google.com/github/GodoyLucas/king_of_rebirth/blob/main/King_of_Rebirth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de desempenho dos players em função do input no campeonado de Call of Duty Warzone King of Rebirth

# Importanto bibliotecas e dados

In [2]:
import pandas as pd
import numpy as np

In [3]:
final = pd.read_csv('https://raw.githubusercontent.com/GodoyLucas/king_of_rebirth/main/king.csv', sep = ";")
final.head()

,player,input,partida,kills,colocacao
0,HARDTREVS,controle,1,5.0,36.0
1,HARDTREVS,controle,2,5.0,12.0
2,HARDTREVS,controle,3,11.0,1.0
3,HARDTREVS,controle,4,3.0,5.0
4,HARDTREVS,controle,5,8.0,7.0


# Verificando condições gerais dos dados

In [4]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   player     352 non-null    object 
 1   input      352 non-null    object 
 2   partida    352 non-null    int64  
 3   kills      348 non-null    float64
 4   colocacao  348 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 13.9+ KB


É possível verificar que, apesar de 4 registros nulos em 'kills' e 'colocacao', as 8 partidas dos 44 players estão presentes no conjunto de dados. 

Os dados nulos são então substituídos por "0" e o tipo dados dos campos são convertidos para inteiro.

In [5]:
final.fillna(value = {'kills': 0, 'colocacao': 0}, inplace = True)
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   player     352 non-null    object 
 1   input      352 non-null    object 
 2   partida    352 non-null    int64  
 3   kills      352 non-null    float64
 4   colocacao  352 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 13.9+ KB


In [6]:
final = final.astype({'kills': int, 'colocacao': int})
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   player     352 non-null    object
 1   input      352 non-null    object
 2   partida    352 non-null    int64 
 3   kills      352 non-null    int64 
 4   colocacao  352 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 13.9+ KB


# Premissas de análise

- A final do campeonato foi realizada em 8 partidas;
- Das 8 partidas, apenas as 5 melhores foram usadas para a classificação geral do campeonato;
- O estudo contempla todas as partidas;
- O campeonato ocorreu no modo solo, ou seja, 1 pessoa por time, competição individual;
- 44 players participaram. Portanto, as colocações nas partidas vão de 1 a 44, não sendo possível empate;
- Nem todos os players participaram e/ou reportaram os resultados de todas as partidas. Houve caso do player não querer jogar, caso de ser desconectado da partida por motivo adverso e caso de não querer reportar o resultado de uma determinada partida. Os resultados faltantes decorrentes desses casos, resultados nulos ou resultados impossíveis dado o contexto serão estimados com base nos resultados do player nas demais partidas.
- Para que a lógica da planilha utilizada para consolidação dos resultados no dia do campeonato não quebrasse, resultados não reportados foram registrados com colocação 44. Este estudo procura identificar os registros legítimos para tratar os que assim só tenham sido reportados pelo uso da planilha.
  - Regristros com colocação 44 com zero kills só são considerados legítimos caso não haja outro registro nesta mesma colocação na partida;
  - Regristros com colocação 44 com mais de zero kills só são considerados legítimos caso não haja outro registro nesta mesma colocação com kills registradas na mesma partida;

Considerando a impossibilidade de empate em qualquer que seja a colocação, o ideal dos resultados seria não haver mais que um player por colocação em cada umas das partidas. Abaixo é possível observar tal ocorrencia, além da ocorrência da colocação "0":

In [43]:
registros_com_mais_de_um_player_por_colocacao = pd.DataFrame()

for partida in range(1, 9):
  for colocacao in range(0, 45):
    if final.query('partida == @partida  & colocacao == @colocacao').player.count() > 1:
      registros_com_mais_de_um_player_por_colocacao = pd.concat([registros_com_mais_de_um_player_por_colocacao, final.query('partida == @partida  & colocacao == @colocacao')])

registros_com_mais_de_um_player_por_colocacao.groupby(['partida', 'colocacao'])[['player']].count()

player
partida colocacao        
1       44              8
2       44              6
3       28              2
        34              2
        44              6
4       0               2
        6               2
        22              2
        44              3
5       0               2
        3               2
        40              2
        44              2
6       19              2
        44              3
7       44             10
8       0               5
        44              7

# Tratamento dos dados

Descrição do roteiro de tratamento dado as regras do jogo e as condições dos dados:

- Identificar e separar registros da colocação "0"
- Identificar e separar registros com mais de um player por colocação por partida;
- Identificar e separar registros legítimos da colocação "44";
- Consolidar registros legítimos sem  necessidade de tratamento;
- Consolidar registros para tratamento;
- Tratar registros;
- Consolidar dados tratados;

## Identificando e separando registros com colocação "0":

In [50]:
registros_com_colocacao_0 = final.query('colocacao == 0')
registros_com_colocacao_0

,player,input,partida,kills,colocacao
18,iErickziN,controle,3,0,0
83,MAUMAU,controle,4,0,0
119,FINEKSS,mouse,8,0,0
179,CLARA FIERCE,controle,4,0,0
199,BALTAFPS,mouse,8,0,0
236,IGOTEE,mouse,5,0,0
237,IGOTEE,mouse,6,0,0
238,IGOTEE,mouse,7,0,0
239,IGOTEE,mouse,8,0,0
255,SAVIOLI,controle,8,0,0


In [59]:
registros_sem_colocacao_0 = final[final.index.isin(list(set(final.index) - set(registros_com_colocacao_0.index)))]
registros_sem_colocacao_0.sort_values('colocacao')

,player,input,partida,kills,colocacao
24,MAIKI,controle,1,10,1
321,FODOVSKI,mouse,2,16,1
2,HARDTREVS,controle,3,11,1
22,iErickziN,controle,7,18,1
13,VIEIRA,controle,6,8,1
...,...,...,...,...,...
74,VOORTZ,mouse,3,0,44
73,VOORTZ,mouse,2,2,44
72,VOORTZ,mouse,1,0,44
103,DETONAMACHO,controle,8,0,44


## Identificando e separando registros com mais de um player por colocação por partida:

In [69]:
registros_com_mais_de_um_player_por_colocacao = pd.DataFrame()

for partida in range(1, 9):
  for colocacao in range(0, 45):
    if registros_sem_colocacao_0.query('partida == @partida  & colocacao == @colocacao').player.count() != 1:
      registros_com_mais_de_um_player_por_colocacao = pd.concat([registros_com_mais_de_um_player_por_colocacao, registros_sem_colocacao_0.query('partida == @partida  & colocacao == @colocacao')])

registros_com_mais_de_um_player_por_colocacao.sort_values('colocacao', ascending = False).head()

,player,input,partida,kills,colocacao
8,VIEIRA,controle,1,0,44
102,DETONAMACHO,controle,7,0,44
331,SAVYTZ,controle,4,1,44
60,RIVOTRIL,mouse,5,0,44
76,VOORTZ,mouse,5,0,44


In [64]:
registros_sem_colocacao_0_e_apenas_um_player_por_colocacao = registros_sem_colocacao_0[registros_sem_colocacao_0.index.isin(list(set(final.index) - set(registros_com_mais_de_um_player_por_colocacao.index)))]
registros_sem_colocacao_0_e_apenas_um_player_por_colocacao.sort_values('colocacao')

,player,input,partida,kills,colocacao
23,iErickziN,controle,8,19,1
2,HARDTREVS,controle,3,11,1
321,FODOVSKI,mouse,2,16,1
219,NICE PADRIN,controle,4,5,1
24,MAIKI,controle,1,10,1
...,...,...,...,...,...
270,KROLPLAYS,controle,7,0,41
209,NINEXT,mouse,2,0,41
292,GARCIA,mouse,5,2,42
131,PRUKKZ,controle,4,1,42


## Identificando e separando registros legítimos da colocação "44":

In [65]:
registros_legitimos_44_colocacao = pd.DataFrame()

for partida in range(1, 9):
  if registros_com_mais_de_um_player_por_colocacao.query('partida == @partida & colocacao == 44 & kills > 0').player.count() == 1:
    registros_legitimos_44_colocacao = pd.concat([registros_legitimos_44_colocacao, registros_com_mais_de_um_player_por_colocacao.query('partida == @partida & colocacao == 44 & kills > 0')])

registros_legitimos_44_colocacao

,player,input,partida,kills,colocacao
73,VOORTZ,mouse,2,2,44
186,YKZHARDCORE,mouse,3,1,44
331,SAVYTZ,controle,4,1,44
350,RAFILD,controle,7,2,44


## Consolidando registros legítimos sem  necessidade de tratamento;

In [68]:
registros_legitimos = pd.concat([registros_legitimos_44_colocacao, registros_sem_colocacao_0_e_apenas_um_player_por_colocacao])
registros_legitimos


,player,input,partida,kills,colocacao
73,VOORTZ,mouse,2,2,44
186,YKZHARDCORE,mouse,3,1,44
331,SAVYTZ,controle,4,1,44
350,RAFILD,controle,7,2,44
0,HARDTREVS,controle,1,5,36
...,...,...,...,...,...
345,RAFILD,controle,2,2,5
346,RAFILD,controle,3,3,15
347,RAFILD,controle,4,7,24
348,RAFILD,controle,5,5,31


## Consolidando registros a serem tratados

In [74]:
registros_a_serem_tratados = final[final.index.isin(list(set(final.index) - set(registros_legitimos.index)))]
registros_a_serem_tratados.sort_values(['partida', 'colocacao'])

,player,input,partida,kills,colocacao
8,VIEIRA,controle,1,0,44
56,RIVOTRIL,mouse,1,0,44
72,VOORTZ,mouse,1,0,44
120,FELPIIS32,mouse,1,0,44
136,ILUDIDO_FPS,controle,1,0,44
...,...,...,...,...,...
95,KID AIMBOT,controle,8,0,44
103,DETONAMACHO,controle,8,0,44
135,PRUKKZ,controle,8,5,44
167,DREYK,controle,8,5,44


## Tratando dados

In [75]:
base_para_tratamento = registros_legitimos[registros_legitimos.player.isin(list(registros_a_serem_tratados.player.unique()))].groupby(by = 'player')[['kills', 'colocacao']].mean().sort_values(by = 'kills', ascending = False)
base_para_tratamento

,kills,colocacao
player,,
iErickziN,11.571429,15.428571
VIEIRA,8.333333,11.333333
SROGATA,7.000000,21.000000
NICE PADRIN,6.333333,10.000000
DREYK,5.833333,18.666667
FODOVSKI,5.833333,13.666667
ANXIETY,5.714286,22.142857
MAUMAU,5.571429,9.428571
KID AIMBOT,5.200000,14.400000


In [76]:
registros_tratados = pd.merge(registros_a_serem_tratados, base_para_tratamento, how = 'inner', on = 'player')
registros_tratados.drop(['kills_x', 'colocacao_x'], axis = 1, inplace = True)
registros_tratados.rename(columns={'kills_y': 'kills', 'colocacao_y': 'colocacao'}, inplace = True)
registros_tratados

,player,input,partida,kills,colocacao
0,VIEIRA,controle,1,8.333333,11.333333
1,VIEIRA,controle,2,8.333333,11.333333
2,iErickziN,controle,3,11.571429,15.428571
3,SURILOCAO,mouse,3,2.000000,24.000000
4,SURILOCAO,mouse,7,2.000000,24.000000
5,EUCHAVS,controle,4,4.500000,16.666667
6,EUCHAVS,controle,7,4.500000,16.666667
7,VOORTZ,mouse,1,4.500000,22.750000
8,VOORTZ,mouse,3,4.500000,22.750000
9,VOORTZ,mouse,5,4.500000,22.750000


## Consolidando registros para análise

In [77]:
final_tratado = pd.concat([registros_legitimos, registros_tratados])
final_tratado

,player,input,partida,kills,colocacao
73,VOORTZ,mouse,2,2.000000,44.000000
186,YKZHARDCORE,mouse,3,1.000000,44.000000
331,SAVYTZ,controle,4,1.000000,44.000000
350,RAFILD,controle,7,2.000000,44.000000
0,HARDTREVS,controle,1,5.000000,36.000000
...,...,...,...,...,...
54,FODOVSKI,mouse,8,5.833333,13.666667
55,SAVYTZ,controle,2,4.333333,13.666667
56,SAVYTZ,controle,3,4.333333,13.666667
57,ANXIETY,mouse,3,5.714286,22.142857


# Analise

In [79]:
final_tratado.groupby(by = 'input')[['kills', 'colocacao']].mean()

,kills,colocacao
input,,
controle,4.678068,18.849634
mouse,3.369398,22.178922


In [80]:
registros_legitimos.groupby(by = 'input')[['kills', 'colocacao']].mean()

,kills,colocacao
input,,
controle,4.702857,18.931429
mouse,3.436364,21.781818


In [81]:
final.groupby(by = 'input')[['kills', 'colocacao']].mean()

,kills,colocacao
input,,
controle,4.100962,20.807692
mouse,2.708333,24.229167
